In [1]:
import numpy as np
import tensorflow as tf

for g in tf.config.list_physical_devices("GPU"):
    tf.config.experimental.set_memory_growth(g, True)

In [7]:
from tensorflow.python.ops.resource_variable_ops import VariableSpec
from tensorflow.python.distribute import values as value_lib

shape = (3, 3)

def create_value(ctx):
    return tf.Variable(tf.random.normal(shape=shape))
    # return tf.random.normal(shape=shape)

def create_tensor(ctx):
    return tf.random.normal(shape=shape)

strategy = tf.distribute.MirroredStrategy()
v = strategy.experimental_distribute_values_from_function(create_value)
t = strategy.experimental_distribute_values_from_function(create_tensor)
assert isinstance(v, value_lib.PerReplica)

@tf.function(input_signature=[VariableSpec(shape=(3, 3), dtype=tf.float32, name='v')])
def my_print(v):
    tf.print(v)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')


In [8]:
from tensorflow.python.distribute import distribute_utils

In [17]:
s = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')


In [38]:
def my_run(t):
    ctx = tf.distribute.get_replica_context()
    return ctx.merge_call(lambda ctx, v: tf.concat(v.values, axis=0), args=(t,)) + 0.000000001 * float(ctx.replica_id_in_sync_group)

In [39]:
ret = s.run(my_run, args=(t,))

In [49]:
[1, -1] + list(v.values[0].shape[:-1])

[1, -1, 3]